# Hawkes Modeling (with `tick` if available)
We attempt to fit a Hawkes process; if `tick` is not present, we fallback to interarrival analysis.

In [ ]:
import numpy as np, matplotlib.pyplot as plt
np.random.seed(7)
T=5000
# simulate timestamps with bursts
lam0=0.8
ll=[]; t=0.0
while len(ll)<T:
    t += np.random.exponential(1/lam0)
    if np.random.rand()<0.05:
        for _ in range(np.random.poisson(5)):
            t += np.random.exponential(0.1)
            ll.append(t)
    else:
        ll.append(t)
arr=np.array(ll)


## 1. Try `tick` fit

In [ ]:
try:
    from tick.hawkes import HawkesExpKern
    model=HawkesExpKern(decays=1.0)
    model.fit([arr])
    print('baseline', model.baseline)
    ks=np.linspace(0,5,200)
    # no direct plot to comply with plain matplotlib constraint
    plt.figure(); plt.hist(np.diff(arr), bins=50); plt.title('Interarrivals (with tick fit)'); plt.show()
except Exception as e:
    # fallback
    ia=np.diff(arr)
    plt.figure(); plt.hist(ia, bins=50); plt.title('Interarrivals (fallback)'); plt.show()


## 2. Q-Q plot vs exponential

In [ ]:
ia=np.diff(arr)
ia=ia[ia>0]
emp=np.sort(ia)
import numpy as np
lam=1/np.mean(emp)
uni=(np.arange(1,len(emp)+1)-0.5)/len(emp)
exp=-np.log(1-uni)/lam
plt.figure(); plt.plot(exp, emp, '.'); plt.plot(exp,exp,'-'); plt.title('QQ plot'); plt.show()


## 3. Autocorrelation of arrivals

In [ ]:
x=(ia-np.mean(ia))/np.std(ia)
acf=[np.corrcoef(x[:-k],x[k:])[0,1] for k in range(1,50)]
plt.figure(); plt.stem(range(1,50), acf, use_line_collection=True); plt.title('ACF'); plt.show()


## 4-10. Experiments (burstiness, bins, windowed stats)

In [ ]:
def window_stats(ts, w=50):
    out=[]; arr=np.array(ts)
    for i in range(w, len(arr)):
        out.append(np.mean(np.diff(arr[i-w:i])))
    return np.array(out)
ws=window_stats(arr)
plt.figure(); plt.plot(ws); plt.title('Windowed mean interarrival'); plt.show()
